In [1]:
import json
import torch

from datasets import load_dataset

from ram import inference_ram_openset as inference
from ram import get_transform
from ram.models import ram_plus

c:\Users\pierc\miniconda3\envs\CS263-MLLM\Lib\site-packages\fairscale\experimental\nn\offload.py:19: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_fwd(orig_func)  # type: ignore
c:\Users\pierc\miniconda3\envs\CS263-MLLM\Lib\site-packages\fairscale\experimental\nn\offload.py:30: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_bwd(orig_func)  # type: ignore


### Load POPE benchmark dataset

In [2]:
category = 'adversarial'

dataset = load_dataset("lmms-lab/POPE", "default")
dataset = dataset['test'].filter(lambda x: x['category'] == category)
dataset

Filter:   0%|          | 0/9000 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'question_id', 'question', 'answer', 'image_source', 'image', 'category'],
    num_rows: 3000
})

In [14]:
def generate_image_tags(dataset):

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print('Device:', device)

    img_size = 384
    model_weights = "pretrained/ram_plus_swin_large_14m.pth"
    output_path = f"{category}_tags.json"
    with open(output_path, 'w') as f:
        f.write('')

    transform = get_transform(image_size=img_size)

    model = ram_plus(pretrained=model_weights,
                    image_size=img_size,
                    vit='swin_l'
            )
    
    model.eval()
    model = model.to(device)

    prev_img_source = None
    for idx in range(len(dataset)):
        img = dataset['image'][idx]
        img_source = dataset['image_source'][idx]

        # Skip duplicate images
        if img_source == prev_img_source:
            continue
        prev_img_source = img_source

        img = transform(img).unsqueeze(0).to(device)

        tags = inference(img, model)
        
        with open(output_path, 'a') as f:
            json.dump({img_source: tags}, f)
            f.write(',\n')

        print(img_source, ": ", tags)

    print('Tags generated and saved to', output_path)


In [ ]:
generate_image_tags(dataset)